In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

# Parameters
POPULATION_SIZE = 50
GENES = 10  # Length of hexadecimal chromosome
GEN_COUNT = 30  # Set to 30 generations
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.05

# Define possible attack types
ATTACK_TYPES = [
    "Denial of Service (DoS)",
    "Distributed Denial of Service (DDoS)",
    "Man-in-the-Middle (MITM)",
    "Packet Sniffing",
    "SQL Injection"
]

def display_attack_type():
    attack_type = random.choice(ATTACK_TYPES)
    print("\n" + "="*50)
    print(f"Attack Detected: {attack_type}")
    print("="*50 + "\n")
    return attack_type

# Generate a random chromosome
def generate_chromosome():
    return ''.join(random.choices('0123456789ABCDEF', k=GENES))

# Diverse initialization of population with varied fitness
def diverse_initialize_population(size):
    population = set()
    while len(population) < size:
        population.add(generate_chromosome())
    return list(population)

# Initialize population with diversity in fitness for first 10 generations
def initialize_population(size, generation):
    if generation <= 10:
        return diverse_initialize_population(size)
    else:
        return [generate_chromosome() for _ in range(size)]

# Fitness function
def fitness(chromosome):
    detection_rate = int(chromosome[:4], 16) / 65535  # Convert hex to decimal
    resilience = int(chromosome[4:8], 16) / 65535
    response_time = int(chromosome[8:], 16) / 255
    return max(0, detection_rate * 0.5 + resilience * 0.3 - response_time * 0.2 + 2.0)

# Roulette Wheel Selection
def roulette_wheel_selection(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    probabilities = [f / total_fitness for f in fitness_scores]
    selected = set()
    while len(selected) < 26:
        selected.add(np.random.choice(population, p=probabilities))
    return list(selected)

# Rank-Based Selection
def rank_selection(population, fitness_scores):
    ranked_pop = sorted(zip(population, fitness_scores), key=lambda x: x[1], reverse=True)
    selected = set(chrom for chrom, _ in ranked_pop[:26])
    return list(selected)

# Crossover
def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        point = random.randint(1, GENES - 1)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]
    return parent1, parent2

# Mutation - Modified to guarantee mutation
def mutate(chromosome):
    idx = random.randint(0, GENES - 1)
    original_gene = chromosome[idx]
    possible_genes = [gene for gene in '0123456789ABCDEF' if gene != original_gene]
    mutated_gene = random.choice(possible_genes)
    return chromosome[:idx] + mutated_gene + chromosome[idx + 1:]

# Evolve population
def evolve_population(population, generation, fitness_over_generations, print_generations):
    if generation <= 10:
        fitness_scores = [fitness(chrom) for chrom in population]
        if generation in print_generations:
            print(f"Generation {generation} Fitness Scores:")
            for idx, (chrom, score) in enumerate(zip(population, fitness_scores)):
                print(f"Chromosome {idx + 1}: {chrom}, Fitness: {score:.4f}")
        selected = roulette_wheel_selection(population, fitness_scores)
        if generation in print_generations:
            print("\nSelected Chromosomes (Roulette Wheel Selection):")
    else:
        ranked_pop = [(chrom, fitness(chrom)) for chrom in population]
        ranked_pop.sort(key=lambda x: x[1], reverse=True)
        if generation in print_generations:
            print(f"\nGeneration {generation} Ranked Chromosomes:")
            for rank, (chrom, score) in enumerate(ranked_pop, 1):
                print(f"Rank {rank}: Chromosome: {chrom}, Fitness: {score:.4f}")
        fitness_scores = [score for _, score in ranked_pop]
        selected = [chrom for chrom, _ in ranked_pop[:26]]
        if generation in print_generations:
            print("\nSelected Chromosomes (Rank Selection):")

    for idx, chrom in enumerate(selected):
        if generation in print_generations:
            print(f"Chromosome {idx + 1}: {chrom}")

    print("\nPerforming Crossover on selected chromosomes:")
    next_population = set()
    crossover_results = []

    for i in range(0, len(selected), 2):
        if i + 1 < len(selected):
            parent1, parent2 = selected[i], selected[i + 1]
            child1, child2 = crossover(parent1, parent2)
            if generation in print_generations:
                print(f"Parents {i+1} & {i+2}: {parent1} + {parent2}")
                print(f"Children: {child1}, {child2}")
            crossover_results.extend([child1, child2])

    if generation in print_generations:
        print("\nAfter Crossover:")
    for i, chromosome in enumerate(crossover_results):
        if generation in print_generations:
            print(f"Chromosome {i+1}: {chromosome}")

    next_population.update(crossover_results)

    print("\nPerforming Mutation on 2 random chromosomes:")
    # Modified mutation section to avoid KeyError
    mutation_candidates = list(next_population)[:2]  # Take first 2 chromosomes from next_population
    mutated_chromosomes = []

    for i, chromosome in enumerate(mutation_candidates):
        if chromosome in next_population:  # Check if chromosome exists before removing
            next_population.remove(chromosome)
        mutated = mutate(chromosome)
        while mutated in next_population:  # Ensure uniqueness
            mutated = mutate(chromosome)
        mutated_chromosomes.append(mutated)
        if generation in print_generations:
            print(f"Chromosome {i+1}: Original: {chromosome} -> Mutated: {mutated}")
        next_population.add(mutated)

    selected_fitness = [(chrom, fitness(chrom)) for chrom in selected]
    selected_fitness.sort(key=lambda x: x[1], reverse=True)

    print("\nSelecting remaining chromosomes from fittest selected parents:")

    for chrom, fit_val in selected_fitness:
        if len(next_population) >= POPULATION_SIZE:
            break
        if chrom not in next_population:
            next_population.add(chrom)
            if generation in print_generations:
                print(f"Added parent chromosome: {chrom}, Fitness: {fit_val:.4f}")

    while len(next_population) < POPULATION_SIZE:
        new_chrom = generate_chromosome()
        if new_chrom not in next_population:
            next_population.add(new_chrom)
            if generation in print_generations:
                print(f"Added new random chromosome: {new_chrom}")

    if generation == GEN_COUNT:
        print("\nFinal Population (Ranked by Fitness):")
        final_population = list(next_population)[:POPULATION_SIZE]
        # Calculate fitness for each chromosome and create ranked list
        ranked_final = [(chrom, fitness(chrom)) for chrom in final_population]
        ranked_final.sort(key=lambda x: x[1], reverse=True)  # Sort by fitness in descending order

        # Print ranked chromosomes with their fitness values
        for rank, (chrom, fit_val) in enumerate(ranked_final, 1):
            print(f"Rank {rank}: Chromosome: {chrom}, Fitness: {fit_val:.4f}")

        # Store the best solution for later use
        best_solution = ranked_final[0][0]
        print(f"\nBest Solution Selected:")
        print(f"Chromosome: {best_solution}, Fitness: {ranked_final[0][1]:.4f}")
    else:
        print("\nNew Population for Next Generation:")
        final_population = list(next_population)[:POPULATION_SIZE]
        for idx, chrom in enumerate(final_population):
            if generation in print_generations:
                print(f"Chromosome {idx + 1}: {chrom}")

    max_fitness = max(fitness(chrom) for chrom in final_population)
    fitness_over_generations.append(max_fitness)

    return final_population, fitness_over_generations

def simulate_attack(chromosome):
    detection_rate = int(chromosome[:4], 16) / 65535  # First part: detection rate (0 to 1)
    resilience = int(chromosome[4:8], 16) / 65535    # Second part: resilience (0 to 1)
    response_time = int(chromosome[8:], 16) / 255     # Last part: response time (0 to 1)
    defense_effectiveness = detection_rate * 0.5 + resilience * 0.3 - response_time * 0.2
    attack_success = max(0, 1 - defense_effectiveness)  # Attack success is 1 minus defense effectiveness
    return attack_success

# Evaluate the initial population before any genetic operations
def evaluate_initial_population(population):
    print("\nEvaluating Initial Population Attack Success Rates:")
    initial_attack_results = []
    for chrom in population:
        attack_success = simulate_attack(chrom)
        initial_attack_results.append(attack_success)
        print(f"Chromosome: {chrom}, Attack Success Rate: {attack_success:.4f}")
    return initial_attack_results

# Evaluate the attack effectiveness of the best solution from the final population
def evaluate_attack_effectiveness(population, best_solution_chromosome, initial_attack_results):
    best_attack_success = simulate_attack(best_solution_chromosome)
    print("\nEvaluating Attack Effectiveness...\n")
    print(f"Best Chromosome from Final Generation: {best_solution_chromosome}, Attack Success: {best_attack_success:.4f}")

    # Plot the attack success comparison
    plot_attack_comparison(initial_attack_results, best_attack_success)

def plot_attack_comparison(initial_attack_results, best_attack_success):
    plt.figure(figsize=(10, 6))
    plt.hist(initial_attack_results, bins=10, alpha=0.6, color='blue', label='Initial Population Attack Success')
    plt.axvline(x=best_attack_success, color='red', linestyle='dashed', linewidth=2, label=f"Best Solution Attack Success: {best_attack_success:.4f}")
    plt.title('Comparison of Attack Success: Initial Population vs Best Solution')
    plt.xlabel('Attack Success Rate')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_fitness(fitness_over_generations):
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(fitness_over_generations) + 1), fitness_over_generations,
             color='blue', linewidth=2, marker='o', markersize=4)
    plt.xlabel('Generation')
    plt.ylabel('Max Fitness')
    plt.title('Fitness Evolution Over Generations')
    plt.grid(True, linestyle='--', alpha=0.7)

    # Add trend line
    z = np.polyfit(range(1, len(fitness_over_generations) + 1), fitness_over_generations, 1)
    p = np.poly1d(z)
    plt.plot(range(1, len(fitness_over_generations) + 1), p(range(1, len(fitness_over_generations) + 1)),
             "r--", alpha=0.8, label='Trend Line')

    plt.legend()
    plt.tight_layout()
    plt.show()

def genetic_algorithm():
    # Display random attack type at the beginning
    current_attack = display_attack_type()

    population = initialize_population(POPULATION_SIZE, 1)
    fitness_over_generations = []

    # Evaluate initial population before crossover and mutation
    initial_attack_results = evaluate_initial_population(population)

    print_generations = set(range(1, 4))
    print_generations.update(range(GEN_COUNT - 2, GEN_COUNT + 1))

    for generation in range(1, GEN_COUNT + 1):
        print(f"\n{'=' * 20} Generation {generation} {'=' * 20}")
        population, fitness_over_generations = evolve_population(population, generation, fitness_over_generations, print_generations)

    final_fitness_scores = [fitness(chrom) for chrom in population]
    ranked_population = sorted(zip(population, final_fitness_scores), key=lambda x: x[1], reverse=True)

    best_solution = ranked_population[0]
    print(f"\nBest Chromosome in Final Generation: {best_solution[0]}, Fitness: {best_solution[1]:.4f}")

    plot_fitness(fitness_over_generations)

    # Evaluate attack effectiveness of best solution
    evaluate_attack_effectiveness(population, best_solution[0], initial_attack_results)

if __name__ == "__main__":
    genetic_algorithm()